# Salary Prediction from LinkedIn Job Postings - Train XGBoost Model

In [1]:
import xgboost as xgb
import pandas as pd, numpy as np
import salary
from sklearn.pipeline import make_pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from category_encoders import TargetEncoder, OneHotEncoder
from sklearn.impute import SimpleImputer

/Users/nginyc/repos/job_posting_salary_prediction/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Train & Evaluate Model

In [10]:
(X_train, y_train) = salary.get_train_dataset()
(X_test, y_test) = salary.get_test_dataset()

### Use Target Encoded Norm Title

In [27]:
model = make_pipeline(
    ColumnTransformer(
        transformers=[
            ('target_encoder', TargetEncoder(), ['norm_title', 'location_state', 'company_industries']),
            ('one_hot_encoder', OneHotEncoder(use_cat_names=True), ['clustered_edu_req', 'clustered_pref_qual']),
            ('experience_level', salary.experience_level_encoder, ['formatted_experience_level']),
            ('work_type', salary.work_type_encoder, ['formatted_work_type']),
            ('remote_allowed', 'passthrough', ['remote_allowed']),
            ('company_employee_count', SimpleImputer(strategy='median'), ['company_employee_count']),
        ],
        remainder='drop'
    ),
    StandardScaler(),
    xgb.XGBRegressor(
        n_estimators=1000,
        max_depth=5,
        learning_rate=0.1,
        verbosity=2,
        reg_lambda=1e2,
        objective='reg:squarederror',
        subsample=0.8,
        random_state=42
    )
).fit(X_train, y_train)

salary.evaluate_train_predictions(model.predict(X_train))

[03:39:44] INFO: /Users/runner/work/xgboost/xgboost/src/data/iterative_dmatrix.cc:53: Finished constructing the `IterativeDMatrix`: (27885, 32, 892320).
Train R2: 0.6221
Train RMSE: 37813.5032
Train MAE: 23757.7600


In [28]:
salary.evaluate_test_predictions(model.predict(X_test))

Test R2: 0.5298
Test RMSE: 39975.5421
Test MAE: 26313.5332
On average, our predicted salaries are $26313.53 off the true salaries
This is 39.40% better than a naive global mean


In [12]:
def get_important_features(model, n=10):
    feature_names = model[:-1].get_feature_names_out(salary.df_X.columns)
    coef = model[-1].feature_importances_
    idxs = np.flip(np.argsort(np.abs(coef))[-n:])
    pd.set_option('display.max_colwidth', None)
    return pd.DataFrame([feature_names[idxs], coef[idxs]], index=["Feature", "Coefficient"]).transpose()

get_important_features(model)

,Feature,Coefficient
0,one_hot_encoder__clustered_edu_req_High school diploma or equivalent,0.140358
1,one_hot_encoder__clustered_edu_req_High school diploma/GED/equivalent preferred,0.093004
2,experience_level__formatted_experience_level,0.091174
3,target_encoder__norm_title,0.05329
4,target_encoder__company_industries,0.042287
5,"one_hot_encoder__clustered_edu_req_Bachelor's degree in Computer Science, Information Systems, Computer Engineering or related field",0.038735
6,one_hot_encoder__clustered_edu_req_nan,0.038673
7,one_hot_encoder__clustered_edu_req_Graduate of an accredited program and board certified.,0.036136
8,one_hot_encoder__clustered_pref_qual_nan,0.028979
9,one_hot_encoder__clustered_pref_qual_Healthcare training/experience.,0.027178


### Use One Hot Encoded Norm Title


In [25]:
model = make_pipeline(
    ColumnTransformer(
        transformers=[
            ('one_hot_encoder', OneHotEncoder(use_cat_names=True), ['norm_title', 'clustered_edu_req', 'clustered_pref_qual']),
            ('target_encoder', TargetEncoder(), ['location_state', 'company_industries']),
            ('experience_level', salary.experience_level_encoder, ['formatted_experience_level']),
            ('work_type', salary.work_type_encoder, ['formatted_work_type']),
            ('remote_allowed', 'passthrough', ['remote_allowed']),
            ('company_employee_count', SimpleImputer(strategy='median'), ['company_employee_count']),
        ],
        remainder='drop'
    ),
    StandardScaler(),
    xgb.XGBRegressor(
        n_estimators=1000,
        max_depth=5,
        learning_rate=0.1,
        verbosity=2,
        reg_lambda=1e2,
        objective='reg:squarederror',
        subsample=0.8,
        random_state=42
    )
).fit(X_train, y_train)

salary.evaluate_train_predictions(model.predict(X_train))

[03:39:26] INFO: /Users/runner/work/xgboost/xgboost/src/data/iterative_dmatrix.cc:53: Finished constructing the `IterativeDMatrix`: (27885, 93, 2593305).
Train R2: 0.6104
Train RMSE: 38395.6739
Train MAE: 24265.4596


In [26]:
salary.evaluate_test_predictions(model.predict(X_test))

Test R2: 0.5350
Test RMSE: 39753.2256
Test MAE: 26169.9725
On average, our predicted salaries are $26169.97 off the true salaries
This is 39.73% better than a naive global mean


In [16]:
get_important_features(model)

,Feature,Coefficient
0,experience_level__formatted_experience_level,0.048066
1,one_hot_encoder__clustered_edu_req_High school diploma or equivalent,0.047244
2,one_hot_encoder__norm_title_Administrative Assistant,0.036594
3,one_hot_encoder__clustered_edu_req_High school diploma/GED/equivalent preferred,0.033407
4,one_hot_encoder__norm_title_Senior Software Engineer,0.023981
5,one_hot_encoder__norm_title_Customer Service Representative,0.023723
6,"one_hot_encoder__clustered_edu_req_Bachelor's degree in Computer Science, Information Systems, Computer Engineering or related field",0.022913
7,target_encoder__company_industries,0.022558
8,one_hot_encoder__norm_title_Medical Assistant,0.021083
9,one_hot_encoder__norm_title_Nurse Practitioner,0.019383
